In [1]:
from openslide import OpenSlide
from PIL import Image
import numpy as np
import os
from time import time
from natsort import natsorted

In [2]:
def svs2tiff(svs,rsf):
    src,fn = os.path.split(svs)
    fn,ext = os.path.splitext(fn)
    fn1 = fn + '.tif'
    if os.path.exists(os.path.join(svs_dst,fn1)): return
    print(fn)
    svs_obj = OpenSlide(svs)
    svs_img = svs_obj.read_region(location=(0,0),level=0,size=svs_obj.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf/float(svs_obj.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(svs_obj.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(svs_obj.dimensions[0]/resize_factorx)),int(np.ceil(svs_obj.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(svs_dst,fn1),resolution=1,resolution_unit=1,quality=100,compression=None)

In [3]:
svs_src = r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\c2'
svs_dst = r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\c2\1um'
if not os.path.exists(svs_dst):
    os.mkdir(svs_dst)
svss = [os.path.join(svs_src,_) for _ in os.listdir(svs_src) if _.endswith(('ndpi','svs')) ]
svss = natsorted(svss)

In [4]:
st = time()
for idx,svs in enumerate(svss):
    # try:
    svs2tiff(svs,1)
    print(idx,'/',len(svss))
    # except:
    #     print('error in the following image')
    #     print(svs)
    #     continue

print("{:.2f} sec elapsed for {:d} images at 10x".format(time()-st,len(svss)))

1C2
0 / 34
4C2
1 / 34
7C2
2 / 34
10C2
3 / 34
13C2
4 / 34
16C2
5 / 34
19C2
6 / 34
22C2
7 / 34
25C2
8 / 34
28C2
9 / 34
31C2
10 / 34
34C2
11 / 34
37C2
12 / 34
40C2
13 / 34
43C2
14 / 34
46C2
15 / 34
49C2
16 / 34
17 / 34
18 / 34
19 / 34
20 / 34
21 / 34
22 / 34
23 / 34
24 / 34
25 / 34
26 / 34
27 / 34
28 / 34
29 / 34
30 / 34
31 / 34
32 / 34
33 / 34
9728.21 sec elapsed for 34 images at 10x
